# SED Fitting using GALAPY

## Import Libraries

In [ ]:
import numpy as np

from astropy.io import fits
from astropy.table import Table

from galapy.internal.utils import cat_to_dict
import galapy.analysis.plot as gplot
from galapy.analysis.plot import plt
from galapy.internal.utils import cat_to_dict

# Import and prepare catalogs 

In [1]:
filters = ['CTIO.DECAM.g', 'CTIO.DECAM.r', 'CTIO.DECAM.i', 'CTIO.DECAM.z', 'CTIO.DECAM.y', 'Paranal.VISTA.Y_filter', 'Paranal.VISTA.J_filter', 'Paranal.VISTA.H_filter', 'Paranal.VISTA.Z_filter', 'WISE.W1', 'WISE.W2']
combined_filters = filters + [f"{filter}_err" for filter in filters]

print(combined_filters)

['CTIO.DECAM.g', 'CTIO.DECAM.r', 'CTIO.DECAM.i', 'CTIO.DECAM.z', 'CTIO.DECAM.y', 'Paranal.VISTA.Y_filter', 'Paranal.VISTA.J_filter', 'Paranal.VISTA.H_filter', 'Paranal.VISTA.Z_filter', 'WISE.W1', 'WISE.W2', 'CTIO.DECAM.g_err', 'CTIO.DECAM.r_err', 'CTIO.DECAM.i_err', 'CTIO.DECAM.z_err', 'CTIO.DECAM.y_err', 'Paranal.VISTA.Y_filter_err', 'Paranal.VISTA.J_filter_err', 'Paranal.VISTA.H_filter_err', 'Paranal.VISTA.Z_filter_err', 'WISE.W1_err', 'WISE.W2_err']


In [ ]:
# print available columns
print(emu_g23_df.columns)

# print flux columns
columns = emu_g23_df.columns
columns_with_flux = [col for col in columns if ('flux' in col) and ('err' in col) and ('G23' in col) and (col[-1:] != 'l')]
print(columns_with_flux) 


In [ ]:
emu_g23_df['G23_CATAID'][13]

In [ ]:
columns_with_flux = ['G23_flux_FUVt', 'G23_flux_NUVt', 'G23_flux_ut', 'G23_flux_gt', 'G23_flux_rt', 'G23_flux_it', 'G23_flux_Zt', 'G23_flux_Yt', 'G23_flux_Jt', 'G23_flux_Ht', 'G23_flux_Kt', 'G23_flux_W1t', 'G23_flux_W2t']
columns_with_error = ['G23_flux_err_FUVt', 'G23_flux_err_NUVt', 'G23_flux_err_ut', 'G23_flux_err_gt', 'G23_flux_err_rt', 'G23_flux_err_it', 'G23_flux_err_Zt', 'G23_flux_err_Yt', 'G23_flux_err_Jt', 'G23_flux_err_Ht', 'G23_flux_err_Kt', 'G23_flux_err_W1t', 'G23_flux_err_W2t']
filters = ['GALEX.FUV', 'GALEX.NUV', 'SDSS.u', 'SDSS.g', 'SDSS.r', 'SDSS.i', 'SDSS.z', 'Paranal.VISTA.Y_filter', 'Paranal.VISTA.J_filter', 'Paranal.VISTA.H_filter', 'Paranal.VISTA.Ks_filter', 'WISE.W1', 'WISE.W2']

column_list_to_convert = columns_with_flux + columns_with_error

## Convert Jansky to MiliJansky
Convert all columns from a given list from Jansky to Milijansky

In [ ]:
jansky_columns = []

In [ ]:
def convert_to_millijansky(fits_file, column_list):
    with fits.open(fits_file) as hdul:
        data = hdul[1].data
        columns = data.columns.names 

        for column_name in column_list:
            if column_name in columns:
                data[column_name] = data[column_name] * 1000
                print(f"Converted {column_name} values to Millijansky.")
            else:
                print(f"Column '{column_name}' not found in the FITS file.")

        hdul.writeto(fits_file.replace('.fits', '_mjy.fits'), overwrite=True)
        print(f"Converted data saved to {fits_file.replace('.fits', '_mjy.fits')}")
        return
    
convert_to_millijansky(emu_g23_catalog_path, column_list_to_convert )

## GALAPY code

In [ ]:
from galapy.PhotometricSystem import print_filters
print_filters()

In [ ]:
def print_row_values(row_number, column_list, fits_file):
    with fits.open(fits_file) as hdul:
        data = hdul[1].data
        if row_number < 0 or row_number >= len(data):
            print(f"Row number {row_number} is out of range.")
            return
        
        row = data[row_number]
        # Convert numpy.float to normal Python float
        values = [float(row[column]) if isinstance(row[column], (np.float32, np.float64)) else row[column]
                  for column in column_list if column in data.columns.names]
        print(values)


print_row_values(20, columns_with_error, emu_g23_mlj_catalog_path)

In [ ]:
catalogue = cat_to_dict(
    emu_g23_mlj_catalog_path, id_field='name', err_field='_err',
    meta_fields = ['redshift', 'redshift_err'],
    skip_fields = ['ra', 'dec', 'semimaj_arcsec', 'axial_ratio', 'pos_angle', '*_flag']
)



# Run SED Fitting

## Get and plot the Results

In [ ]:
from galapy.sampling.Results import load_results

res = load_results('/home/polaris/Lab_Astro/data/344680520004824_emcee_results_light.galapy.hdf5')

model = res.get_model()
observation = res.get_observation()
handler = res.get_handler()
print('The run has ', res.Ndof, 'degrees of freedom')

In [ ]:
res.get_stored_quantities()

In [ ]:
print(f"Best-fitting stellar mass: {res.get_bestfit('Mstar'):.2e} Msol")
print(f"Median stellar mass: {res.get_median('Mstar'):.2e} Msol")
print(res.get_bestfit('Mdust'))
print(res.get_bestfit('Zstar'))
print(res.get_bestfit('Zgas'))
print(res.get_bestfit('SFR'))
print(res.get_bestfit('TMC'))
print(res.get_bestfit('TDD'))


In [ ]:
# If plot_contours=True it breaks the code

fig, axes = plt.subplots(2,1,figsize=(6,5),tight_layout=True,
                                      sharex = True,
                         gridspec_kw={'height_ratios':(4.5,1.5), 'hspace':0.0})

###################################################
# Plot the SED:

_ = gplot.sed_flux_res(
    res, plot_components=True, plot_observation=True, plot_contours=False,
    ax = axes[0], # passing the first Axis to the ax argument
    ax_kwargs = {
       'xlim':(1.e+2, 1.e+10),
       'ylim':(1.e-10,1.e+8),
    },
)

###################################################
# Plot the residuals

_ = gplot.sed_residuals_res(
    res, frame='obs', plot_contours=False, plot_chi2 = True,
    ax = axes[1], # passing the second Axis to the ax argument
    text_kwargs={'loc':'lower right'},
    ax_kwargs={
       'xlim':(1.e+2, 1.e+10),
       'ylim':(-2.5e+2,2.5e+2),
    },
)

